In [1]:
import pandas as pd
import pickle
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import warnings
from sklearn.metrics import (
    recall_score,
    precision_score,
    f1_score,
    accuracy_score,
    roc_auc_score,
)

# Ignore the warning message
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# baseline model
def build_model():
    model = Sequential()
    model.add(Dense(24, input_dim=24, activation="relu"))
    model.add(Dense(48, activation="relu"))
    model.add(Dense(24, activation="relu"))
    model.add(Dense(12, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    return model

In [3]:
metrics_DL = pd.DataFrame(
    columns=["Sentence", "Accuracy", "Precision", "Recall", "F1", "AUC"]
)

n = list(range(2, 61))

feature_cols = [
    "Erstloesung",
    "Schussel",
    "Erfolg",
    "Schwierigkeit",
    "ist_Schulzeit",
    "MehrfachFalsch",
    "vorher_abgebrochen",
    "Fehler",
    "Klassenstufe",
    "Jahredabei",
    "AnzahlAufgaben",
    "Sex__m",
    "Sex__w",
    "Testposition__pruefung",
    "Testposition__training",
    "Testposition__version",
    "Art__GK",
    "Art__GR",
    "Art__GZ",
    "Art__K",
    "Art__LB",
    "UserAttribut",
    "OrderNumber",
    "steps",
]

In [ ]:
for i in n:
    path = "matrices_allsessions/matrix" + str(i) + ".pkl"
    infile = open(path, "rb")
    df = pickle.load(infile)
    infile.close()
    df = df.reset_index()

    y_len = len(feature_cols)
    X = df[feature_cols].astype(float)
    y = df.y
    y = y.astype("int")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=1
    )

    model = build_model()

    model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

    model.fit(
        x=X_train,
        y=y_train,
        epochs=10,
        batch_size=128,
        verbose=0,
        validation_data=(X_test, y_test),
    )

    scores = model.evaluate(x=X_test, y=y_test, verbose=0)
    # print(scores)

    yhat_probs = model.predict(X_test, verbose=0)
    yhat_classes = (model.predict(X_test) > 0.5).astype("int32")
    # reduce to 1d array
    yhat_probs = yhat_probs[:, 0]
    yhat_classes = yhat_classes[:, 0]

    accuracy = accuracy_score(y_test, yhat_classes)
    precision = precision_score(y_test, yhat_classes)
    recall = recall_score(y_test, yhat_classes)
    f1 = f1_score(y_test, yhat_classes)
    auc = roc_auc_score(y_test, yhat_probs)

    metrics_DL = metrics_DL.append(
        {
            "Sentence": i,
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1": f1,
            "AUC": auc,
        },
        ignore_index=True,
    )

metrics_DL.to_pickle("model_metrics/metrics_DL.pkl")

In [5]:
# # save sample prediction
# pred = model.predict(X_test)
# pred = 1 - pred
# data_df = pred.tolist()
# data_df = pd.DataFrame(data_df)
# data_df.to_pickle("DL55.pkl")